# This Notebook is for the Coursera Capstone Project.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
CLIENT_ID = 'GLZNC2UC2YULIFBDCCDKHEQVRZ3PEDKSLYLAG3HYINOXFBYH' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: GLZNC2UC2YULIFBDCCDKHEQVRZ3PEDKSLYLAG3HYINOXFBYH


In [22]:
# type your answer here
LIMIT = 500 # Maximum is 100
cities = ["Karachi, Pakistan", 'Lahore, Pakistan', 'Islamabad, Pakistan', 'Quetta, Pakistan', 'Peshawar, Pakistan']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d1ca941735") # PIZZA PLACE CATEGORY ID
    results[city] = requests.get(url).json()

In [23]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [38]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of pizza places in {city} = ", results[city]['response']['totalResults'])
    print("Showing all those that can be found")

Total number of pizza places in Karachi, Pakistan =  66
Showing all those that can be found
Total number of pizza places in Lahore, Pakistan =  61
Showing all those that can be found
Total number of pizza places in Islamabad, Pakistan =  31
Showing all those that can be found
Total number of pizza places in Quetta, Pakistan =  5
Showing all those that can be found
Total number of pizza places in Peshawar, Pakistan =  5
Showing all those that can be found


In [6]:
maps[cities[0]]


In [7]:
maps[cities[1]]


In [25]:
maps[cities[2]]


In [26]:
maps[cities[3]]


In [27]:
maps[cities[4]]


We can see that Karachi and Lahore are the most dense cities with Pizza places. 

Let's do a concrete measure of this density.

For this analysis will be using basic statistics. I will get the mean location of the pizza places which should be near to most of them if they are really dense or far if not.

After that we will take the average of the distance of the venues to the mean coordinates

In [28]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

Karachi, Pakistan
Mean Distance from Mean coordinates
0.05486709369660373
Lahore, Pakistan
Mean Distance from Mean coordinates
0.04374081932652691
Islamabad, Pakistan
Mean Distance from Mean coordinates
0.043883464383447535
Quetta, Pakistan
Mean Distance from Mean coordinates
0.022110723875260245
Peshawar, Pakistan
Mean Distance from Mean coordinates
0.024414660137189063


In [29]:
maps[cities[0]]


In [30]:
maps[cities[1]]


In [31]:
maps[cities[2]]


In [32]:
maps[cities[3]]


In [33]:
maps[cities[4]]


### We now see that KARACHI has the largest amount of pizza places followed by LAHORE and lastly by at third place is ISLAMABAD. Our tourist's best interest would be to book a hotel near that mean coordinate to surround themselves with all these pizza restaurant!!
Another observation is that there is one really far away Pizza store which would possible increase its score in Islamabad, as Islamabad is a small city compared to Karachi and Lahore its mean distance form mean co-ordinate should be less than that of previous 2 big cities  let's try to remove it and calculate it again

In [34]:
city = 'Islamabad, Pakistan'
venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 

print(city)
print("Mean Distance from Mean coordinates")
dists = np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)
dists.sort()
print(np.mean(dists[:-1]))# Ignore the biggest distance

Islamabad, Pakistan
Mean Distance from Mean coordinates
0.03826952786458182


The Corrected mean distance has now decrease by 0.01 point which shows that pizza places are near in Islamabad.

## With this I conclude My notebook and Thanks Coursera for providing a great learning platform.